In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Load dataset
df = pd.read_csv("tmdb_with_moods.csv")

# Preprocess
df['overview'] = df['overview'].fillna('')
df['genres'] = df['genres'].fillna('')
df['release_date'] = df['release_date'].fillna('')
df['mood'] = df['mood'].fillna('')
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year.fillna(0).astype(int)
df['combined_features'] = df['overview'] + " " + df['genres'] + " " + df['mood']

# TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_features'])

# Movie recommendation function
def recommend_movies(input_year, input_mood, top_n=10):
    input_vec = vectorizer.transform([input_mood])
    cosine_sim = cosine_similarity(input_vec, tfidf_matrix).flatten()

    # Filter exact year
    filtered_df = df[df['year'] == input_year]
    if filtered_df.empty:
        print(f"😕 No movies found in {input_year}. Looking for nearby years...")

        # Try previous year
        prev_df = df[df['year'] == input_year - 1]
        next_df = df[df['year'] == input_year + 1]
        extended_df = pd.concat([prev_df, next_df])

        if extended_df.empty:
            return "❌ No movies found in nearby years either."

        print(f"✅ Found {len(extended_df)} movies in year {input_year - 1} or {input_year + 1}")
        year_indices = extended_df.index
        scores = cosine_sim[year_indices]
        top_indices = scores.argsort()[::-1][:top_n]
        return extended_df.iloc[top_indices][['title', 'genres', 'year', 'overview']].reset_index(drop=True)

    # Found in exact year
    year_indices = filtered_df.index
    scores = cosine_sim[year_indices]
    top_indices = scores.argsort()[::-1][:top_n]
    return filtered_df.iloc[top_indices][['title', 'genres', 'year', 'overview']].reset_index(drop=True)

# 🌟 Get user input
try:
    year_input = int(input("📅 Enter the year of movie you want to watch: "))
    mood_input = input("🧠 Enter your current mood (e.g., happy, romantic, thrilling): ")

    print("\n🔍 Recommending movies based on your mood and year...\n")
    results = recommend_movies(year_input, mood_input)

    if isinstance(results, str):
        print(results)
    else:
        for idx, row in results.iterrows():
            print(f"{idx+1}. {row['title']} ({row['year']})")
            print(f"   Genre: {row['genres']}")
            print(f"   Overview: {row['overview'][:250]}...\n")

except ValueError:
    print("⚠️ Please enter a valid year.")


📅 Enter the year of movie you want to watch:  2006
🧠 Enter your current mood (e.g., happy, romantic, thrilling):  happy



🔍 Recommending movies based on your mood and year...

1. Phat Girlz (2006)
   Genre: ['Comedy', 'Romance']
   Overview: Two plus-sized ladies meet the men of their dreams in the most unexpected of ways....

2. Ira & Abby (2006)
   Genre: ['Comedy', 'Romance']
   Overview: A neurotic, young psychology student, with low self-esteem, has a chance encounter with a free-spirited, extremely gregarious woman who works at the Paris Health Club in New York City, and who suggests that they immediately get married to see how it ...

3. Friends with Money (2006)
   Genre: ['Comedy', 'Drama', 'Romance']
   Overview: After she quits her lucrative job, Olivia finds herself unsure about her future and her relationships with her successful and wealthy friends....

4. The Shaggy Dog (2006)
   Genre: ['Comedy', 'Family']
   Overview: The tale of a workaholic dad-turned-dog who finds that being man's best friend shows him the most important job - being a great dad....

5. Failure to Launch (2006)
   Genr

In [ ]:
#front_end
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from kivy.uix.scrollview import ScrollView
from kivy.uix.gridlayout import GridLayout
from kivy.uix.spinner import Spinner
from kivy.core.window import Window

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Set app background color
Window.clearcolor = (0.1, 0.1, 0.1, 1)

# Load and preprocess dataset
df = pd.read_csv("tmdb_with_moods.csv")
df.fillna({'overview': '', 'genres': '', 'release_date': '', 'mood': ''}, inplace=True)
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year.fillna(0).astype(int)
df['combined_features'] = df['overview'] + " " + df['genres'] + " " + df['mood']

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_features'])

# Unique moods from the dataset
mood_options = sorted(list(df['mood'].unique()))

def recommend_movies(year, mood, top_n=10):
    input_vec = vectorizer.transform([mood])
    similarity_scores = cosine_similarity(input_vec, tfidf_matrix).flatten()

    year_filtered = df[df['year'] == year]
    if year_filtered.empty:
        nearby_years = df[df['year'].isin([year - 1, year + 1])]
        if nearby_years.empty:
            return ["⚠ No movies found for the given year or nearby."]
        indices = nearby_years.index
        scores = similarity_scores[indices]
        top_indices = scores.argsort()[::-1][:top_n]
        return [
            f"{row['title']} ({row['year']})\nGenre: {row['genres']}\nOverview: {row['overview'][:300]}..."
            for _, row in nearby_years.iloc[top_indices].iterrows()
        ]

    indices = year_filtered.index
    scores = similarity_scores[indices]
    top_indices = scores.argsort()[::-1][:top_n]
    return [
        f"{row['title']} ({row['year']})\nGenre: {row['genres']}\nOverview: {row['overview'][:300]}..."
        for _, row in year_filtered.iloc[top_indices].iterrows()
    ]

class StyledLabel(Label):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.color = (1, 1, 1, 1)
        self.font_size = kwargs.get("font_size", 18)
        self.bold = True
        self.size_hint_y = None
        self.height = kwargs.get("height", 30)

class MovieRecommenderUI(BoxLayout):
    def __init__(self, **kwargs):
        super().__init__(orientation='vertical', padding=20, spacing=15, **kwargs)

        self.add_widget(StyledLabel(text='🎬 Movie Recommendation System', font_size=26, height=50))

        self.add_widget(StyledLabel(text='Enter Release Year:'))
        self.year_input = TextInput(
            multiline=False, size_hint_y=None, height=40,
            background_color=(0.2, 0.2, 0.2, 1), foreground_color=(1, 1, 1, 1),
            hint_text="e.g. 2015"
        )
        self.add_widget(self.year_input)

        self.add_widget(StyledLabel(text='Select Mood:'))
        self.mood_spinner = Spinner(
            text="Choose a mood",
            values=mood_options,
            size_hint_y=None,
            height=40,
            background_color=(0.3, 0.3, 0.3, 1),
            color=(1, 1, 1, 1)
        )
        self.add_widget(self.mood_spinner)

        self.recommend_button = Button(
            text='🎥 Recommend Movies', background_color=(1, 0.3, 0.4, 1),
            color=(1, 1, 1, 1), size_hint_y=None, height=50
        )
        self.recommend_button.bind(on_press=self.show_recommendations)
        self.add_widget(self.recommend_button)

        self.scroll_view = ScrollView(size_hint=(1, 1))
        self.results_box = GridLayout(cols=1, spacing=10, size_hint_y=None, padding=10)
        self.results_box.bind(minimum_height=self.results_box.setter('height'))
        self.scroll_view.add_widget(self.results_box)
        self.add_widget(self.scroll_view)

    def show_recommendations(self, _):
        self.results_box.clear_widgets()
        try:
            year = int(self.year_input.text.strip())
            mood = self.mood_spinner.text

            if mood == "Choose a mood":
                raise ValueError("No mood selected.")

            results = recommend_movies(year, mood)
            for movie in results:
                label = StyledLabel(text=movie, font_size=16)
                label.text_size = (self.width - 40, None)
                label.bind(texture_size=label.setter('size'))
                self.results_box.add_widget(label)

        except ValueError:
            self.results_box.add_widget(StyledLabel(
                text="⚠ Please enter a valid year and select a mood.",
                font_size=16
            ))

class MovieApp(App):
    def build(self):
        return MovieRecommenderUI()

if __name__ == '__main__':
    MovieApp().run()

In [ ]:
#back_end
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from kivy.uix.scrollview import ScrollView
from kivy.uix.gridlayout import GridLayout
from kivy.uix.image import Image
from kivy.core.window import Window
from kivy.uix.widget import Widget
from kivy.uix.behaviors import ButtonBehavior
from kivy.uix.floatlayout import FloatLayout

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Set app background color
Window.clearcolor = (0.1, 0.1, 0.1, 1)

# Load and preprocess the dataset
df = pd.read_csv("tmdb_cleaned_dataset-use.csv")
df['overview'] = df['overview'].fillna('')
df['genres'] = df['genres'].fillna('')
df['release_date'] = df['release_date'].fillna('')
df['mood'] = df['mood'].fillna('')
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year.fillna(0).astype(int)
df['combined_features'] = df['overview'] + " " + df['genres'] + " " + df['mood']

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_features'])

# Recommendation function
def recommend_movies(input_year, input_mood, top_n=10):
    input_vec = vectorizer.transform([input_mood])
    cosine_sim = cosine_similarity(input_vec, tfidf_matrix).flatten()
    
    filtered_df = df[df['year'] == input_year]
    if filtered_df.empty:
        prev_df = df[df['year'] == input_year - 1]
        next_df = df[df['year'] == input_year + 1]
        extended_df = pd.concat([prev_df, next_df])
        if extended_df.empty:
            return ["No movies found in nearby years either."]
        year_indices = extended_df.index
        scores = cosine_sim[year_indices]
        top_indices = scores.argsort()[::-1][:top_n]
        return [f"{row['title']} ({row['year']}): {row['genres']}" for _, row in extended_df.iloc[top_indices].iterrows()]

    year_indices = filtered_df.index
    scores = cosine_sim[year_indices]
    top_indices = scores.argsort()[::-1][:top_n]
    return [f"{row['title']} ({row['year']}): {row['genres']}" for _, row in filtered_df.iloc[top_indices].iterrows()]

# Custom styled label
class StyledLabel(Label):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.color = (1, 1, 1, 1)
        self.font_size = 20
        self.bold = True

class MovieRecommender(BoxLayout):
    def __init__(self, **kwargs):
        super().__init__(orientation='vertical', padding=20, spacing=15, **kwargs)

        self.add_widget(StyledLabel(text='🎬 Movie Recommendation System', font_size=28))

        self.add_widget(StyledLabel(text='Enter the Year:'))
        self.year_input = TextInput(multiline=False, size_hint_y=None, height=40, background_color=(0.2, 0.2, 0.2, 1), foreground_color=(1, 1, 1, 1))
        self.add_widget(self.year_input)

        self.add_widget(StyledLabel(text='Enter your Mood:'))
        self.mood_input = TextInput(multiline=False, size_hint_y=None, height=40, background_color=(0.2, 0.2, 0.2, 1), foreground_color=(1, 1, 1, 1))
        self.add_widget(self.mood_input)

        self.submit_btn = Button(text='🎥 Recommend Movies', background_color=(1, 0, 0.3, 1), color=(1, 1, 1, 1), size_hint_y=None, height=50)
        self.submit_btn.bind(on_press=self.get_recommendations)
        self.add_widget(self.submit_btn)

        self.results_box = GridLayout(cols=1, spacing=10, size_hint_y=None, padding=[10])
        self.results_box.bind(minimum_height=self.results_box.setter('height'))

        self.scroll = ScrollView(size_hint=(1, 1))
        self.scroll.add_widget(self.results_box)
        self.add_widget(self.scroll)

    def get_recommendations(self, instance):
        self.results_box.clear_widgets()
        try:
            year = int(self.year_input.text)
            mood = self.mood_input.text.strip()
            results = recommend_movies(year, mood)
            for movie in results:
                self.results_box.add_widget(StyledLabel(text=movie, font_size=16))
        except ValueError:
            self.results_box.add_widget(StyledLabel(text="Please enter a valid year and mood.", font_size=16))

class MovieApp(App):
    def build(self):
        return MovieRecommender()

if __name__ == '__main__':
    MovieApp().run()
